In [1]:
import pandas as pd
import numpy as np

from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

import os
import sys
import warnings
warnings.filterwarnings("ignore")
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesse
 
from dataset import engine, RFM, Churn, Engagement

In [2]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine


def create_db(user="root", password="Chenlu1974", server="localhost", database="transact"):
    SQLALCHEMY_DATABASE_URL = "mysql+pymysql://{}:{}@{}/{}".format(
        user, password, server, database
    )
    engine = create_engine(SQLALCHEMY_DATABASE_URL)

    SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
    Base = declarative_base()

    return engine, SessionLocal, Base


engine, SessionLocal, Base = create_db(password='msql1234')

In [4]:
with engine.connect() as db:
	query = sqlalchemy.text(
		'''
		SELECT * FROM campaign;
		''')
	df = pd.DataFrame(db.execute(query).fetchall())
	db.close()

df

,campaign_id,campaign_name,start_date,end_date,target_segment,budget,channel,goal,displays
0,0001,Campaign_1,2023-01-09,2023-01-21,High-income,561.54,search,conversion,369796
1,0002,Campaign_2,2023-09-03,2023-10-03,Retirees,195.14,influencer,conversion,187097
2,0003,Campaign_3,2023-04-10,2023-04-29,Retirees,193.24,influencer,awareness,184686
3,0004,Campaign_4,2024-02-11,2024-02-18,High-income,121.95,app,conversion,139851
4,0005,Campaign_5,2024-12-07,2025-01-03,Young Adults,526.39,email,conversion,364506
...,...,...,...,...,...,...,...,...,...
295,0296,Campaign_296,2024-12-04,2025-01-03,High-income,318.59,social,consideration,254233
296,0297,Campaign_297,2023-02-06,2023-02-18,Young Adults,151.76,sms,retention,97410
297,0298,Campaign_298,2024-09-15,2024-10-10,Young Adults,139.23,email,awareness,137429
298,0299,Campaign_299,2024-12-11,2024-12-29,Retirees,175.80,app,retention,195941


In [5]:
rfm = RFM(engine)
rfm = rfm.get_dataset().loc[:, ['customer_id', 'segment']]

In [7]:
with engine.connect() as db:
	query = sqlalchemy.text(
		'''
		SELECT * FROM engagement;
		''')
	df = pd.DataFrame(db.execute(query).fetchall())
	db.close()
eng = df.loc[:, ['customer_id', 'action_type']]

In [ ]:
merged = eng.merge(rfm, on='customer_id') \

merged 

,customer_id,action_type,segment
0,0916,scrolled,New Customers
1,1651,scrolled,Hibernating
2,1874,scrolled,Hibernating
3,0892,converted,Hibernating
4,1368,scrolled,Potential Loyalists
...,...,...,...
78391,1395,converted,At Risk
78392,0760,clicked,Potential Loyalists
78393,1271,clicked,Champions
78394,1918,scrolled,Loyal Customers


In [ ]:
with engine.connect() as db:
	query = sqlalchemy.text(
		'''
		SELECT * FROM engagement;
		''')
	df = pd.DataFrame(db.execute(query).fetchall())
	db.close()
data = df[df['engagement_date'] >= df['engagement_date'].max() - relativedelta(months=11)] \
	.groupby([df['engagement_date'].dt.to_period("M"), 'action_type']) \
	.agg(['count'])['customer_id'] \
	.reset_index() \
	.pivot(index="engagement_date", columns="action_type", values="count") \
	.reset_index() \
	.loc[:, ['engagement_date', 'converted', 'credentials', 'clicked', 'scrolled']] \
	.rename(columns={'engagement_date': 'date'}) \
	.astype({'date': str})
# df
data['convertedColor'] = ["hsl(229, 70%, 50%)" for i in range(len(data))]
data['credentialsColor'] = ["hsl(296, 70%, 50%)" for i in range(len(data))]
data['clickedColor'] = ["hsl(97, 70%, 50%)" for i in range(len(data))]
data['scrolledColor'] = ["hsl(229, 70%, 50%)" for i in range(len(data))]


result = data.to_dict(orient='records')
print(result)

[{'date': '2024-01', 'converted': 15, 'credentials': 18, 'clicked': 48, 'scrolled': 102, 'convertedColor': 'hsl(229, 70%, 50%)', 'credentialsColor': 'hsl(296, 70%, 50%)', 'clickedColor': 'hsl(97, 70%, 50%)', 'scrolledColor': 'hsl(229, 70%, 50%)'}, {'date': '2024-02', 'converted': 219, 'credentials': 325, 'clicked': 1242, 'scrolled': 2465, 'convertedColor': 'hsl(229, 70%, 50%)', 'credentialsColor': 'hsl(296, 70%, 50%)', 'clickedColor': 'hsl(97, 70%, 50%)', 'scrolledColor': 'hsl(229, 70%, 50%)'}, {'date': '2024-03', 'converted': 158, 'credentials': 303, 'clicked': 1001, 'scrolled': 2020, 'convertedColor': 'hsl(229, 70%, 50%)', 'credentialsColor': 'hsl(296, 70%, 50%)', 'clickedColor': 'hsl(97, 70%, 50%)', 'scrolledColor': 'hsl(229, 70%, 50%)'}, {'date': '2024-04', 'converted': 128, 'credentials': 255, 'clicked': 920, 'scrolled': 1612, 'convertedColor': 'hsl(229, 70%, 50%)', 'credentialsColor': 'hsl(296, 70%, 50%)', 'clickedColor': 'hsl(97, 70%, 50%)', 'scrolledColor': 'hsl(229, 70%, 50%)'

In [4]:
# fig, ax = plt.subplots(figsize=(10, 8))
# df_new.plot(kind='line', stacked=True, linestyle='-', marker='o', ax=ax)
# plt.legend()

In [5]:
with engine.connect() as db:
	query = sqlalchemy.text(
		'''
		SELECT *
		FROM engagement e
		ORDER BY e.engagement_date DESC
		LIMIT 10;
		''')
	df = pd.DataFrame(db.execute(query).fetchall())
	db.close()

df['engagement_date'] = df['engagement_date'].astype(str)
df = df.loc[:, ['customer_id', 'engagement_date', 'action_type', 'feedback_score']]
df = df.rename(columns={'customer_id': 'id', 'engagement_date': 'date', 'action_type': 'action', 'feedback_score': 'score'})
# df.to_dict(orient='records')
df

,id,date,action,score
0,1904,2024-12-28,scrolled,2
1,1071,2024-12-28,credentials,1
2,1660,2024-12-28,scrolled,2
3,0134,2024-12-28,scrolled,1
4,1805,2024-12-28,scrolled,1
5,0130,2024-12-28,clicked,3
6,1931,2024-12-28,scrolled,2
7,1048,2024-12-28,credentials,2
8,1679,2024-12-28,scrolled,2
9,1435,2024-12-28,clicked,3


In [6]:
with engine.connect() as db:
	query = sqlalchemy.text(
		'''
		SELECT * 
		FROM campaign c;
		''')
	df = pd.DataFrame(db.execute(query).fetchall())
	db.close()
data = df[df['start_date'] >= df['start_date'].max() - relativedelta(months=11)] \
	.groupby([df['start_date'].dt.to_period("M")]) \
	.agg({"budget": ['sum']})['budget'].reset_index().rename(columns={'start_date':'date', 'sum':'spending'})
data['date'] = data['date'].astype(str)
data['spending'] = data['spending'].astype(int)
data['dataColor'] = ["hsl(229, 70%, 50%)" for i in range(len(data))]
data.to_dict(orient='records')

[{'date': '2024-01', 'spending': 1499, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-02', 'spending': 1286, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-03', 'spending': 1842, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-04', 'spending': 544, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-05', 'spending': 1196, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-06', 'spending': 3810, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-07', 'spending': 2193, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-08', 'spending': 1255, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-09', 'spending': 1726, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-10', 'spending': 2506, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-11', 'spending': 5039, 'dataColor': 'hsl(229, 70%, 50%)'},
 {'date': '2024-12', 'spending': 7629, 'dataColor': 'hsl(229, 70%, 50%)'}]